In [41]:
!pip install openai

In [42]:
import pandas as pd
import plotly.express as px
from openai import OpenAI
from google.colab import userdata
from dataclasses import dataclass, field

In [43]:
api_key = userdata.get('openai_api_key')
openai = OpenAI(api_key=api_key)
model = 'gpt-4o-2024-05-13'

In [ ]:
@dataclass
class BossAgent:

    start_word: str = 'こんにちは'
    model = model
    openai = openai
    messages: list = field(default_factory=list)

    @property
    def system_call(self):
        system_call = '''
        あなたは優秀なデータ分析課の上司です。解答を急がず、
        部下の分析をより広い視野から発送できる、データからよい意思決定を導ける、課題を深堀できる、
        データからエビデンスを導ける、指示やアドバイスを出してください。
        '''.strip()
        return system_call

    def chat(self, prompt: str):
        if self.messages == []:
            sys = self.system_call + f'最初の指示: {self.start_word}'
            self.messages.append({'role': 'system', 'content': sys})
        self.messages.append({'role': 'user', 'content': prompt})
        try:
            res = openai.chat.completions.create(
                model=self.model,
                messages=self.messages,
            )
            self.messages.append({'role': 'assistant', 'content': res.choices[0].message.content})
            return res.choices[0].message.content
        except Exception as e:
            print(e)
            return None


@dataclass
class DataAnalyst:

    model = model
    openai = openai
    messages: list= field(default_factory=list)

    @property
    def system_call(self):
        system_call = """
        あなたは優秀なデータ分析官です。結論を急がず、自分の考えと
        上司の指示を基に、より課題を深く検討し、
        データ分析を行ってください。

        """
        return system_call


    def analysis(self, prompt: str):
        if self.messages == []:
            self.messages.append({'role': 'system', 'content': self.system_call})
        self.messages.append({'role': 'user', 'content': prompt})
        chat = openai.chat.completions.create(
            model=self.model,
            messages=self.messages
        )
        self.messages.append({'role': 'assistant', 'content': chat.choices[0].message.content})
        return chat.choices[0].message.content

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/work/pycon_kyushu/data/model-eval-price.csv')
start_word = f''' 次のデータから、どのモデルが、プロダクトテストに使うのに割安で良いかを分析してください。
    データ: {df}
'''
boss = BossAgent(start_word)
data_analyst = DataAnalyst()


In [ ]:
res1 = data_analyst.analysis(start_word)

In [ ]:
print(res1)

プロダクトテストに使うのに最もコストパフォーマンスが良いモデルを選ぶためには、パフォーマンス（MMLU、MATH）と価格のバランスを考慮する必要があります。以下の手順で分析を進めます。

### ステップ1: データの確認
データ内の各モデルの情報を確認します。

### ステップ2: 標準化のための指標設定
価格が異なるため、特定の指標を使って各モデルの価格対性能比を比較します。

例えば、各モデルのMMLUとMATHのパフォーマンススコアの合計を元に割安度を算出します。

### パフォーマンス/価格比の計算
それぞれのモデルについて、性能の合計（MMLU + MATH）を価格で割ることで、性能/価格比を求めます。

```plaintext
性能の合計/価格 = (MMLU + MATH) / Price
```

### ステップ3: 比率を計算して比較
比率を計算し、それに基づいて各モデルを比較します。

以下は、計算結果です:

```python
import pandas as pd

# データフレームの作成
data = {
    'model': ['GPT-3.5', 'GPT-4', 'GPT-4 Turbo', 'GPT-4o',
              'Claude 3-Haiku', 'Claude 3-Opus', 'Gemini-1.5-PRO', 'Gemini-1.5-FLASH'],
    'MMLU(5shot)': [70, 86.4, 86.5, 88.7, 75.2, 86.8, 81.9, 78.9],
    'MATH': [34.10, 52.90, 72.60, 76.60, 38.90, 60.10, 58.50, 54.90],
    'Price': [1.5, 60, 30, 15, 1.25, 75, 10.5, 0.53]
}

df = pd.DataFrame(data)

# 性能合計と性能/価格比の計算
df['Performance'] = df['MMLU(5shot)'] + df['MATH']
df['Performance/Price'] = df['Performance'] / df['Price']

# Print the results
print(df

In [ ]:
res2 = boss.chat(res1)
print(res2)

とてもよいアプローチです。データの確認、パフォーマンス/価格比の計算、そして比較をステップごとに行ったことにより、価格対性能比をしっかりと評価できました。では、更に深掘りして、他の要素も考慮してみましょう。

### ステップ 5: 時系列データの考慮
リリース日を考慮すると、新しいモデルは古いものよりも性能が高い傾向があります。また、新しいモデルは今後のサポートやアップデートが期待できるため、リリース日も一つの重要な要素と考えましょう。

```python
# リリース日データを日付型に変換
df['release date'] = pd.to_datetime(df['release date'])

# リリース日からの経過時間（現在の日付を基準に）を計算
current_date = pd.to_datetime('2023/12/31')
df['days_since_release'] = (current_date - df['release date']).dt.days

# 結果を表示
print(df[['model', 'Performance', 'Price', 'Performance/Price', 'days_since_release']])
```

### 追加分析 - コストパフォーマンス指標の加重平均
パフォーマンス/価格比に具体的なウェイト（例：パフォーマンス優先度や価格優先度）を設定してより柔軟な指標を計算します。

```python
# パフォーマンス（MMLUとMATHの加重合計）の重みを設定（例：MMLUをより重要視する）
w_mmlu = 0.7
w_math = 0.3
df['Weighted_Performance'] = df['MMLU(5shot)'] * w_mmlu + df['MATH'] * w_math

# 性能/価格比の計算
df['Weighted_Performance/Price'] = df['Weighted_Performance'] / df['Price']

# 結果を表示
print(df[['model', 'Weighted_Performance', 'Price', 'Weighted_Performance/Price', 'days_since_re

In [ ]:
res3 = data_analyst.analysis(res2)
print(res3)

ご指摘の通り、リリース日やその他の要因を考慮に入れることで、より詳細な分析を行うことができます。以下に追加分析を行いました。

### ステップ 5: 時系列データの考慮
リリース日を基に、新しいモデルがどれくらい新しいかを計算します。

```python
import pandas as pd

# データフレームの作成
data = {
    'model': ['GPT-3.5', 'GPT-4', 'GPT-4 Turbo', 'GPT-4o',
              'Claude 3-Haiku', 'Claude 3-Opus', 'Gemini-1.5-PRO', 'Gemini-1.5-FLASH'],
    'MMLU(5shot)': [70, 86.4, 86.5, 88.7, 75.2, 86.8, 81.9, 78.9],
    'MATH': [34.10, 52.90, 72.60, 76.60, 38.90, 60.10, 58.50, 54.90],
    'Price': [1.5, 60, 30, 15, 1.25, 75, 10.5, 0.53],
    'release_date': ['2022/12', '2023/3/14', '2023/12/1', '2024/5/13', '2024/3/4', '2024/3/4', '2024/2/15', '2024/5/14']
}

df = pd.DataFrame(data)

# リリース日データを日付型に変換
df['release_date'] = pd.to_datetime(df['release_date'])

# リリース日からの経過時間（現在の日付を基準に）を計算
current_date = pd.to_datetime('2023/12/31')
df['days_since_release'] = (current_date - df['release_date']).dt.days

# Performanceの計算
df['Performance'] = df['MMLU(5shot)'] + df['MATH']

# Performance/Priceの計算
df['Performance/Price'] = df

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/work/pycon_kyushu/data/model-eval-price.csv')
start_word = f''' 次のデータから、どのモデルが、データ分析機能を持つアプリケーションのプロダクトテストに使うのに割安で良いかを分析してください。
    データ: {df}
'''
boss = BossAgent(start_word)
data_analyst = DataAnalyst()


In [ ]:
res1 = data_analyst.analysis(start_word)
print(res1)

データを検討する際に考慮すべき要素として、性能（MMLUとMATHのスコア）、価格、およびリリース日があります。それぞれのモデルを比較するために、上記の要素を総合的に評価します。

まず、データ全体を確認して、性能やコストパフォーマンスの観点から有利と思われるモデルを洗い出します。

#### 1. モデル別の性能比較:
- **MMLU**: GPT-4o > Claude 3-Opus > GPT-4 Turbo ≒ GPT-4 > Claude 3-Haiku > Gemini-1.5-PRO > Gemini-1.5-FLASH > GPT-3.5
- **MATH**: GPT-4o > GPT-4 Turbo > Claude 3-Opus > GPT-4 > Gemini-1.5-PRO > Gemini-1.5-FLASH > Claude 3-Haiku > GPT-3.5

#### 2. コストパフォーマンスを考えた評価
各モデルについて、性能と価格のバランスを指数化して評価します。まずは、各モデルの代表的な性能スコア（仮にMMLUとMATHの平均値を採用）を価格で割って、コストパフォーマンスの指標を算出します。

- **GPT-3.5**: (70% + 34.10%) / 2 =  52.05 / 1.50 ≈ 34.70
- **GPT-4**: (86.40% + 52.90%) / 2 = 69.65 / 60.00 ≈ 1.16
- **GPT-4 Turbo**: (86.50% + 72.60%) / 2 = 79.55 / 30.00 ≈ 2.65
- **GPT-4o**: (88.70% + 76.60%) / 2 = 82.65 / 15.00 ≈ 5.51
- **Claude 3-Haiku**: (75.20% + 38.90%) / 2 = 57.05 / 1.25 ≈ 45.64
- **Claude 3-Opus**: (86.80% + 60.10%) / 2 = 73.45 / 75.00 ≈ 0.98
- **Gemini-1.5-PRO**: (81.90% + 58.50%) / 2 = 70.20 / 10.50 ≈ 6.69
- **Gemini-1.5-FLASH**: (78.90% + 

In [ ]:
res2 = boss.chat(res1)
print(res2)

あなたが提示した分析は、性能とコストパフォーマンスのバランスを重視し、モデルを評価するための有効な一手法です。ただし、以下の点をさらに詳細に検討することで、より深い洞察が得られる可能性があります。

1. **性能の重み付け**:
   各アプリケーションにおいて、MMLUとMATHのどちらのスコアがより重要かに応じて、性能評価の重みを変えることが有効です。例えば、数値計算性能が重要なアプリケーションなら、MATHのスコアにより重きを置くべきです。

2. **価格のスケーリング**:
   モデルの導入にかかるコストは、使用頻度や利用規模によって大きく異なります。価格を評価する際に、実際の使用ケース（例えば、月間のAPIコール数）に基づいて検討することが重要です。

3. **リリース日の考慮**:
   最新のモデルは通常改善された機能を提供する可能性がありますが、リリース直後はバグやサポートの問題が発生するリスクも考慮すべきです。この点も場合によっては重要です。

### 追加分析:
以下は、上述のポイントに基づく追加分析の例です：

#### 1. 重み付けを考慮した総合評価：
MMLUとMATHの性能を等しく重視する場合には現行の方法で問題ありませんが、特定の要件に応じた重み付けを適用します（例: MATH 70%、MMLU 30%）。
```python
models = [
    {"name": "GPT-3.5", "MMLU": 70, "MATH": 34.10, "price": 1.50},
    {"name": "GPT-4", "MMLU": 86.40, "MATH": 52.90, "price": 60.00},
    {"name": "GPT-4 Turbo", "MMLU": 86.50, "MATH": 72.60, "price": 30.00},
    {"name": "GPT-4o", "MMLU": 88.70, "MATH": 76.60, "price": 15.00},
    {"name": "Claude 3-Haiku", "MMLU": 75.20, "MATH": 38.90, "price": 1.25},
    {"name": "Claude 3-Opus", "MMLU": 

In [ ]:
res3 = data_analyst.analysis(res2)
print(res3)

あなたの指摘に基づいて、追加分析および詳細な考察を進めます。特に、MMLUとMATHの性能に対して重み付けを行い、実際の運用コスト評価を行い、新しいモデルのリリース日に関する安定性やサポートのリスクについても考慮します。

### 1. 重み付けを考慮した総合評価
たとえば、数値計算性能（MATH）が70%、総合的な知識性能（MMLU）が30%の比率で重要と仮定します。その計算を以下に示します。

```python
models = [
    {"name": "GPT-3.5", "MMLU": 70, "MATH": 34.10, "price": 1.50},
    {"name": "GPT-4", "MMLU": 86.40, "MATH": 52.90, "price": 60.00},
    {"name": "GPT-4 Turbo", "MMLU": 86.50, "MATH": 72.60, "price": 30.00},
    {"name": "GPT-4o", "MMLU": 88.70, "MATH": 76.60, "price": 15.00},
    {"name": "Claude 3-Haiku", "MMLU": 75.20, "MATH": 38.90, "price": 1.25},
    {"name": "Claude 3-Opus", "MMLU": 86.80, "MATH": 60.10, "price": 75.00},
    {"name": "Gemini-1.5-PRO", "MMLU": 81.90, "MATH": 58.50, "price": 10.50},
    {"name": "Gemini-1.5-FLASH", "MMLU": 78.90, "MATH": 54.90, "price": 0.53},
]

weighted_scores = []
for model in models:
    weighted_score = (((model["MMLU"] * 0.3) + (model["MATH"] * 0.7)) / model["price"])
    weighted_scores.append((model["name"], weighted_s

## UPDATE SYSTEM CALL

In [45]:
@dataclass
class BossAgent:

    start_word: str = "こんにちは、私はデータ分析課の上司です。どのようなことでお困りですか？"
    model = model
    openai = openai
    messages: list = field(default_factory=list)

    @property
    def system_call(self):
        system_call = '''
        あなたは優秀なデータ分析課の上司です。
        部下の分析がうまくいくように、以下の点に留意して指示やアドバイスを出してください。

        1. 常に複数の視点からデータを見るよう促す
        2. 分析の目的や課題を明確にするよう助言する
        3. 新しい分析手法やアプローチを提案する
        4. データの限界や注意点について言及する
        5. 分析結果から actionable な洞察を引き出すよう導く
        6. 部下の考えを引き出し、議論を深めるような問いかけを行う
        '''.strip()
        return system_call

    def chat(self, prompt: str):
        if self.messages == []:
            sys = self.system_call + f'最初の指示: {self.start_word}'
            self.messages.append({'role': 'system', 'content': sys})
        self.messages.append({'role': 'user', 'content': prompt})
        try:
            res = openai.chat.completions.create(
                model=self.model,
                messages=self.messages,
            )
            self.messages.append({'role': 'assistant', 'content': res.choices[0].message.content})
            return res.choices[0].message.content
        except Exception as e:
            print(e)
            return None


@dataclass
class DataAnalyst:

    model = model
    openai = openai
    messages: list= field(default_factory=list)

    @property
    def system_call(self):
        system_call = '''
        あなたは優秀なデータ分析官です。上司の指示を基に、以下の点に留意してデータ分析を行ってください。

        1. 様々な角度からデータを観察し、新しい気づきを得る
        2. 分析の目的や課題を常に意識し、フォーカスを維持する
        3. 既存の分析手法に加え、新しいアプローチも積極的に試す
        4. データの特性や制約を理解し、適切に扱う
        5. 分析結果から実務に役立つ洞察を導き出す
        6. 自分の考えを明確に説明し、上司との議論を通じて理解を深める
        '''.strip()
        return system_call


    def analysis(self, prompt: str):
        if self.messages == []:
            self.messages.append({'role': 'system', 'content': self.system_call})
        self.messages.append({'role': 'user', 'content': prompt})
        chat = openai.chat.completions.create(
            model=self.model,
            messages=self.messages
        )
        self.messages.append({'role': 'assistant', 'content': chat.choices[0].message.content})
        return chat.choices[0].message.content

In [46]:
df = pd.read_csv('/content/drive/MyDrive/work/pycon_kyushu/data/model-eval-price.csv')
start_word = f''' 次のデータから、どのモデルが、データ分析機能を持つアプリケーションのプロダクトテストに使うのに割安で良いかを分析してください。
    データ: {df}
'''
boss = BossAgent(start_word)
data_analyst = DataAnalyst()

res1 = data_analyst.analysis(start_word)
print(res1)

このデータから、各モデルのコストパフォーマンスを評価するには、性能（MMLUとMATHのスコア）と価格（Price）を考慮する必要があります。また、リリース日も重要な要素となるでしょうが、ここでは主に性能と価格の観点に焦点を当てます。

### 1. 性能と価格の評価

#### 1.1 性能の統一評価
MMLUとMATHのスコアを統一評価基準にするため、単純平均をとります。
\[ \text{Performance Score} = \frac{\text{MMLU} + \text{MATH}}{2} \]

| Model            | MMLU  | MATH  | Performance Score |
|------------------|-------|-------|-------------------|
| GPT-3.5          | 70.0  | 34.1  | 52.05             |
| GPT-4            | 86.4  | 52.9  | 69.65             |
| GPT-4 Turbo      | 86.5  | 72.6  | 79.55             |
| GPT-4o           | 88.7  | 76.6  | 82.65             |
| Claude 3-Haiku   | 75.2  | 38.9  | 57.05             |
| Claude 3-Opus    | 86.8  | 60.1  | 73.45             |
| Gemini-1.5-PRO   | 81.9  | 58.5  | 70.20             |
| Gemini-1.5-FLASH | 78.9  | 54.9  | 66.90             |

#### 1.2 コストパフォーマンスの評価
各モデルの価格1ドルあたりのパフォーマンススコアを導き出します。
\[ \text{Cost Efficiency} = \frac{\text{Performance Score}}{\text{Price}} \]

| Model            | Price (USD) | 

In [47]:
res2 = boss.chat(res1)
print(res2)

素晴らしい分析です。あなたの考慮点と手順について、具体的かつ明確なステップで進めていると思います。以下に追加の視点や注意点を提供し、さらに深掘りするための問いかけを提示します。

### 追加の分析視点と質問

1. **複数の視点からデータを見る:**
   - **安定性と信頼性**も考慮してはどうでしょうか。特に「リリース日」が最近のものは、新しい技術である一方、問題が安定していない可能性があります。これを重視するかどうかも検討してください。
   - **長期的なコスト効率**についても考える価値があります。あるモデルの初期コストは高いものの、長期的に使用する際の運用コストや更新コストを評価することで、新たな洞察が得られるかもしれません。
   
    質問: **新しいモデルと古いモデルのどちらが安定性や長期的な信頼性で優位かを確認する方法はありますか？**

2. **分析目的の明確化:**
   - 分析の主要目的が「初期コスト最小化」なのか、あるいは「パフォーマンス最大化」なのか、部門全体で合意しておくことが重要です。これにより、選定基準が明確になります。

    質問: **具体的なプロダクトテストのシナリオをもう少し明確にして、その上でモデル選定の基準を再評価する意義はありますか？**

3. **新しい分析手法の提案:**
   - **重み付け平均**を実施することで、MMLUとMATHスコアの評価に対して、ユーザーシナリオや重要ポイントに応じた重みを反映させることができます。
   
    質問: **MMLUとMATHのスコアに対して、どちらに重点を置くべきかについて、ユーザーシナリオに基づく重み付けを試みたことはありますか？**

4. **データの限界や注意点:**
   - 一部のモデルが将来リリースされる予定（例：GPT-4o）の場合、リリース直後の初期データはまだ安定していない可能性があります。これを考慮に入れる必要があります。

    質問: **当該モデルのリリース後のレビューや初期テストデータを追加することで、選定基準が変わる可能性について検討したことはありますか？**

5. **行動可能な洞察の導出:**
   - すでに示していただいた主要な推奨モデルを具体的にどのシナリオで使うか、あるいは異なる仮定（例：テ

In [48]:
res3 = data_analyst.analysis(res2)
print(res3)

ありがとうございます、ご指示と問いかけに基づいてさらに深掘りを行い、追加の視点を取り入れた分析を実施します。

### 1. 安定性と信頼性の評価
新しいモデル（特に最近リリースされたもの）は技術的に優れている可能性がありますが、その一方で安定性や信頼性に欠ける場合があります。そのため、過去の実績やユーザーのフィードバック情報を収集することが重要です。ここでは仮に、リリース日時が新しいものは多少リスクがあると仮定して、統計的分析を進めます。

#### 方法:
- **最新度スコア**: リリース日が遠いほどスコアが減少するように調整
- 最新度スコア = (最大リリース年月日-該当リリース年月日) / 最大リリース年月日

具体的なリリース日が分かる場合には、リリース日に基づく重み付け評価を導入しますが、今回はリリース日データからの相対評価で進みます。

### 2. コストの長期的効果
価格（1M/output）から長期使用を想定して、長期コストを推計します。例えば、1年間使用した場合の総コストを比較します（ここでは単純に12倍して年間コストを計算）。

### 3. 目的の明確化
テストのシナリオが「初期コスト最小化」なのか「パフォーマンス最大化」なのか定義されると、選定基準が明確になります。今回はこれを以下のように仮定します:
- 初期コストと長期コストのバランスを考える
- 高パフォーマンスモデルに重きを置く

### 4. 重み付け平均の計算
目的ごとに異なるシナリオに対して、MMLUとMATHのスコアに重みをつけ、総合評価を行います。
例えば、MMLUとMATHに対して [0.7, 0.3] の重みをつける場合：
\[ \text{Weighted Score} = 0.7 \cdot \text{MMLU} + 0.3 \cdot \text{MATH} \]

### 5. シナリオに基づくシミュレーション
各モデルを特定のシナリオ（ユーザーグループ、予算、使用頻度など）に対してシミュレーションし、モデルの最適性を検証します。

### 具体的なステップ

#### ステップ1: 安定性のための最新度スコア導入

| Model            | Release Date | Stability Score |
|-------------